# Pandas Challenge
## School & Student Data

### Obervations:

- The analysis of this dataset revealed a few unexpected truths:
    - If you look at the _Scores by School Spending_ table you'll see that there's a negative correlation between budget per student and students' scores. It seems the less money per student, the better their test scores and vice versa.
    - In a similar vein, Charter schools appear to have noticeably better scores than District schools. The top 5 schools are all Charter and the bottom 5 are all District.

## Analysis

In [260]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
df = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

In [261]:
# Checking df column datatypes
df.dtypes

Student ID        int64
student_name     object
gender           object
grade            object
school_name      object
reading_score     int64
math_score        int64
School ID         int64
type             object
size              int64
budget            int64
dtype: object

In [262]:
# Check for null values
df.isna().sum()

Student ID       0
student_name     0
gender           0
grade            0
school_name      0
reading_score    0
math_score       0
School ID        0
type             0
size             0
budget           0
dtype: int64

In [263]:
# Gettting a look at the data structure
df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [264]:
# Calculate total number of schools
total_schools = school_data['school_name'].nunique()

In [265]:
# Calculate total number of students
total_students = school_data['size'].sum()

In [266]:
# Calculate total budget
total_budget = school_data['budget'].sum()

In [267]:
# Calculate average math score
average_math_score = df['math_score'].mean()

In [268]:
# Calculate average reading score
average_reading_score = df['reading_score'].mean()

In [269]:
# Calculate percentage of students that have a passing math score
percent_pass_math = (df['math_score'][df.math_score >= 70].count() / len(df['math_score'])) * 100

In [270]:
# Calculate percentage of students that have a passing reading score
percent_pass_reading = (df['reading_score'][df.reading_score >= 70].count() / len(df['reading_score'])) * 100

In [271]:
# Calculate percentage of students that passed overall
percent_pass_overall = (df['Student ID'][(df.reading_score >= 70) & (df.math_score >= 70)].count() / len(df['Student ID']) * 100)

In [272]:
# Create a dataframe to hold the results
district_summary = pd.DataFrame({
    "Total Schools" : total_schools,
    "Total Students": total_students,
    "Total Budget": total_budget,
    "Average Math Score": average_math_score,
    "Average Reading Score": average_reading_score,
    "% Passing Math": percent_pass_math,
    "% Passing Reading": percent_pass_reading,
    "% Overall Passing": percent_pass_overall
}, index = [0])

### District Summary

In [273]:
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463,65.172326


In [274]:
# Create a DataFrame to be joined on by calculations
school_summary = pd.DataFrame({
    "School Name": school_data['school_name'],
    "School Type": school_data['type'],
    "Size": school_data['size'],
    "Total Budget": school_data['budget']
})

In [275]:
# Calculate budget per student
for school in school_data['school_name']:
    budget_per_student = pd.DataFrame({"School Name": school_data['school_name'],
                                       "Budget Per Student" : school_data['budget'] / school_data['size']})

In [276]:
# Create DataFrame of just school names and reading scores
school_reading_average_score = df[['school_name','reading_score']]

# Group this DataFrame by school and aggregate by reading score average
school_reading_average_score = school_reading_average_score.groupby(['school_name']).mean()

# Reset index to default
school_reading_average_score = school_reading_average_score.reset_index()

# Rename column
school_reading_average_score = school_reading_average_score.rename(columns={"school_name": "School Name"})

In [277]:
# Create DataFrame of just school names and math scores
school_math_average_score = df[['school_name','math_score']]

# Group this DataFrame by school and aggregate by math score average
school_math_average_score = school_math_average_score.groupby(['school_name']).mean()

# Reset index to default
school_math_average_score = school_math_average_score.reset_index()

# Rename column
school_math_average_score = school_math_average_score.rename(columns={"school_name": "School Name"})

In [278]:
# Create DataFrame of just school names and reading scores
school_reading_pass_rate = df[['school_name', 'reading_score']]

# Rename school_name to better formatting
school_reading_pass_rate = school_reading_pass_rate.rename(columns={'school_name': 'School Name'})

# Create column counting students
school_reading_pass_rate['student_count'] = 1

# Create column pass/fail for conditional >/< 70
school_reading_pass_rate['pass_count'] = [1 if score > 70 else 0 for score in df['reading_score']]

# Group by school
school_reading_pass_rate = school_reading_pass_rate.groupby('School Name').sum()

# Create pass fail percentage column
school_reading_pass_rate['% Passing Reading'] = (
    school_reading_pass_rate['pass_count'] / school_reading_pass_rate['student_count'] * 100)

# Drop unnecessary columns
school_reading_pass_rate = school_reading_pass_rate.drop(
    ['reading_score','student_count','pass_count'], 1)

# Reset index
school_reading_pass_rate = school_reading_pass_rate.reset_index()

C:\Users\bryantim\AppData\Local\Temp\ipykernel_16624\1119901948.py:21: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  school_reading_pass_rate = school_reading_pass_rate.drop(


In [279]:
# Create DataFrame of just school names and math scores
school_math_pass_rate = df[['school_name', 'math_score']]

# Rename school_name to better formatting
school_math_pass_rate = school_math_pass_rate.rename(columns={'school_name': 'School Name'})

# Create column counting students
school_math_pass_rate['student_count'] = 1

# Create column pass/fail for conditional >/< 70
school_math_pass_rate['pass_count'] = [1 if score > 70 else 0 for score in df['math_score']]

# Group by school
school_math_pass_rate = school_math_pass_rate.groupby('School Name').sum()

# Create pass fail percentage column
school_math_pass_rate['% Passing Math'] = (
    school_math_pass_rate['pass_count'] / school_math_pass_rate['student_count'] * 100)

# Drop unnecessary columns
school_math_pass_rate = school_math_pass_rate.drop(
    ['math_score','student_count','pass_count'], 1)

C:\Users\bryantim\AppData\Local\Temp\ipykernel_16624\3977246586.py:21: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  school_math_pass_rate = school_math_pass_rate.drop(


In [280]:
# Create DataFrame of just school names and reading scores
school_overall_pass_rate = df[['school_name', 'reading_score', 'math_score']]

# Rename school_name to better formatting
school_overall_pass_rate = school_overall_pass_rate.rename(columns={'school_name': 'School Name'})

# Create column counting students
school_overall_pass_rate['student_count'] = 1

# Create reading column pass/fail for conditional >/< 70
school_overall_pass_rate['reading_pass_count'] = [1 if score >= 70 else 0 for score in df['reading_score']]

# Create reading column pass/fail for conditional >/< 70
school_overall_pass_rate['math_pass_count'] = [1 if score >= 70 else 0 for score in df['math_score']]

# Sum up reading and math columns
school_overall_pass_rate['sum_pass_count'] = school_overall_pass_rate['reading_pass_count'] + school_overall_pass_rate['math_pass_count']

# Create overall column pass/fail for conditional = 2
school_overall_pass_rate['overall_pass_count'] = [1 if score == 2 else 0 
                                                  for score in school_overall_pass_rate['sum_pass_count']]

# Group by school
school_overall_pass_rate = school_overall_pass_rate.groupby('School Name').sum()

# Create pass fail percentage column
school_overall_pass_rate['% Passing Overall'] = (
    school_overall_pass_rate['overall_pass_count'] / school_overall_pass_rate['student_count'] * 100)

# Drop unnecessary columns
school_overall_pass_rate = school_overall_pass_rate.drop(
    ['reading_score', 'math_score', 'student_count', 'reading_pass_count', 'math_pass_count',
    'sum_pass_count', 'overall_pass_count'], 1)

C:\Users\bryantim\AppData\Local\Temp\ipykernel_16624\3521200253.py:31: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  school_overall_pass_rate = school_overall_pass_rate.drop(


In [281]:
# Merge dataframes clean up extra columns
school_summary = pd.merge(school_summary, budget_per_student, how='outer', on=["School Name", "School Name"])

In [282]:
# Merge w/ average reading score
school_summary = pd.merge(school_summary, school_reading_average_score, how='outer', on=["School Name", "School Name"])
school_summary = school_summary.rename(columns={"reading_score": "Average Reading Score"})

In [283]:
# Merge w/ average math score
school_summary = pd.merge(school_summary, school_math_average_score, how='outer', on=["School Name", "School Name"])
school_summary = school_summary.rename(columns={"math_score": "Average Math Score"})

In [284]:
# Merge w/ % passing reading
school_summary = pd.merge(school_summary, school_reading_pass_rate, how='outer', on=["School Name", "School Name"])

In [285]:
# Merge w/ % passing math
school_summary = pd.merge(school_summary, school_math_pass_rate, how='outer', on=["School Name", "School Name"])

In [286]:
# Merge w/ % passing overall
school_summary = pd.merge(school_summary, school_overall_pass_rate, how='outer', on=["School Name", "School Name"])

### School Summary

In [287]:
school_summary

,School Name,School Type,Size,Total Budget,Budget Per Student,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,% Passing Overall
0,Huang High School,District,2917,1910635,655.0,81.182722,76.629414,78.813850,63.318478,53.513884
1,Figueroa High School,District,2949,1884411,639.0,81.158020,76.711767,78.433367,63.750424,53.204476
2,Shelton High School,Charter,1761,1056600,600.0,83.725724,83.359455,92.617831,89.892107,89.892107
3,Hernandez High School,District,4635,3022020,652.0,80.934412,77.289752,78.187702,64.746494,53.527508
4,Griffin High School,Charter,1468,917500,625.0,83.816757,83.351499,93.392371,89.713896,90.599455
5,Wilson High School,Charter,2283,1319574,578.0,83.989488,83.274201,93.254490,90.932983,90.582567
6,Cabrera High School,Charter,1858,1081356,582.0,83.975780,83.061895,93.864370,89.558665,91.334769
7,Bailey High School,District,4976,3124928,628.0,81.033963,77.048432,79.300643,64.630225,54.642283
8,Holden High School,Charter,427,248087,581.0,83.814988,83.803279,92.740047,90.632319,89.227166
9,Pena High School,Charter,962,585858,609.0,84.044699,83.839917,92.203742,91.683992,90.540541


In [288]:
# Create a DataFrame of the top 5 rows by % Passing Overall
top_5_overall_passing = school_summary.nlargest(5,'% Passing Overall')
top_5_overall_passing.sort_values(by=['% Passing Overall'], inplace=True, ascending=False)

### Top Performing Schools (By % Overall Passing)

In [289]:
top_5_overall_passing

,School Name,School Type,Size,Total Budget,Budget Per Student,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,% Passing Overall
6,Cabrera High School,Charter,1858,1081356,582.0,83.975780,83.061895,93.864370,89.558665,91.334769
14,Thomas High School,Charter,1635,1043130,638.0,83.848930,83.418349,92.905199,90.214067,90.948012
4,Griffin High School,Charter,1468,917500,625.0,83.816757,83.351499,93.392371,89.713896,90.599455
5,Wilson High School,Charter,2283,1319574,578.0,83.989488,83.274201,93.254490,90.932983,90.582567
9,Pena High School,Charter,962,585858,609.0,84.044699,83.839917,92.203742,91.683992,90.540541


In [290]:
# Create a DataFrame of the bottom 5 rows by % Passing Overall
bottom_5_overall_passing = school_summary.nsmallest(5,'% Passing Overall')
bottom_5_overall_passing.sort_values(by=['% Passing Overall'], inplace=True)

### Bottom Performing Schools (By % Overall Passing)

In [291]:
bottom_5_overall_passing

,School Name,School Type,Size,Total Budget,Budget Per Student,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,% Passing Overall
11,Rodriguez High School,District,3999,2547363,637.0,80.744686,76.842711,77.744436,64.066017,52.988247
1,Figueroa High School,District,2949,1884411,639.0,81.158020,76.711767,78.433367,63.750424,53.204476
0,Huang High School,District,2917,1910635,655.0,81.182722,76.629414,78.813850,63.318478,53.513884
3,Hernandez High School,District,4635,3022020,652.0,80.934412,77.289752,78.187702,64.746494,53.527508
12,Johnson High School,District,4761,3094650,650.0,80.966394,77.072464,78.281874,63.852132,53.539172


In [292]:
# Create DataFrame w/ just school, math scores, and grade
math_scores_by_grade = df[['school_name', 'math_score', 'grade']]

# Rename columns
math_scores_by_grade = math_scores_by_grade.rename(columns=
                                                   {'school_name':'School Name', 'math_score': 'Math Score', 'grade': 'Grade'})

# Create a pivot table of math scores by school and grade using math scores as values
math_scores_by_grade = pd.pivot_table(
    math_scores_by_grade, values = 'Math Score', index=['School Name'], columns = 'Grade')

# Reorder the columns so they are chronological
math_scores_by_grade = math_scores_by_grade[['9th', '10th', '11th', '12th']]

### Math Scores by Grade

In [293]:
math_scores_by_grade

Grade,9th,10th,11th,12th
School Name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [294]:
# Create DataFrame w/ just school, reading scores, and grade
reading_scores_by_grade = df[['school_name', 'reading_score', 'grade']]

# Rename columns
reading_scores_by_grade = reading_scores_by_grade.rename(columns=
                                                   {'school_name':'School Name', 'reading_score': 'Reading Score', 'grade': 'Grade'})

# Create a pivot table of math scores by school and grade using math scores as values
reading_scores_by_grade = pd.pivot_table(
    reading_scores_by_grade, values = 'Reading Score', index=['School Name'], columns = 'Grade')

# Reorder the columns so they are chronological
reading_scores_by_grade = reading_scores_by_grade[['9th', '10th', '11th', '12th']]

### Reading Scores by Grade

In [295]:
reading_scores_by_grade

Grade,9th,10th,11th,12th
School Name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [296]:
# Create DataFrame of just necessary columns from school_summary
scores_by_school_spending = school_summary[
    ['Budget Per Student','Average Math Score', 'Average Reading Score', '% Passing Math', 
     '% Passing Reading', '% Passing Overall' ]]

In [297]:
# Create bins for spending per student ranges
bins = [0, 584, 629, 644, 680]

group_labels = ['<$585', '$585-630', '$630-645', '$645-680']

In [298]:
# Slice the data and place it into bins
pd.cut(scores_by_school_spending['Budget Per Student'], bins, labels=group_labels).head()

0    $645-680
1    $630-645
2    $585-630
3    $645-680
4    $585-630
Name: Budget Per Student, dtype: category
Categories (4, object): ['<$585' < '$585-630' < '$630-645' < '$645-680']

In [299]:
 # Place the data series into a new column inside of the DataFrame
scores_by_school_spending["Spending Per Student Group"] = pd.cut(scores_by_school_spending['Budget Per Student'], bins, labels=group_labels)

C:\Users\bryantim\AppData\Local\Temp\ipykernel_16624\3414563277.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scores_by_school_spending["Spending Per Student Group"] = pd.cut(scores_by_school_spending['Budget Per Student'], bins, labels=group_labels)


In [300]:
# Group by spending per student group
scores_by_school_spending = scores_by_school_spending.groupby('Spending Per Student Group').mean()

# Drop budget per student column
scores_by_school_spending = scores_by_school_spending[
    ['Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', '% Passing Overall']]

###  Scores by School Spending

In [301]:
scores_by_school_spending

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Spending Per Student Group,,,,,
<$585,83.455399,83.933814,90.350436,93.325838,90.369459
$585-630,81.899826,83.155286,83.980055,89.378647,81.418596
$630-645,78.518855,81.624473,70.946108,81.648261,62.857656
$645-680,76.997210,81.027843,63.972368,78.427809,53.526855


In [302]:
# Create DataFrame of just necessary columns from school_summary
scores_by_school_size = school_summary[
    ['Size','Average Math Score', 'Average Reading Score', '% Passing Math', 
     '% Passing Reading', '% Passing Overall' ]]

In [303]:
# Create bins for school sizes 
bins = [0, 999, 1999, 5000]

group_labels = ['Small (< 1000)', 'Medium (1000-2000)', 'Large (2000-5000)']

In [304]:
# Slice the data and place it into bins
pd.cut(scores_by_school_size['Size'], bins, labels=group_labels)

0      Large (2000-5000)
1      Large (2000-5000)
2     Medium (1000-2000)
3      Large (2000-5000)
4     Medium (1000-2000)
5      Large (2000-5000)
6     Medium (1000-2000)
7      Large (2000-5000)
8         Small (< 1000)
9         Small (< 1000)
10    Medium (1000-2000)
11     Large (2000-5000)
12     Large (2000-5000)
13     Large (2000-5000)
14    Medium (1000-2000)
Name: Size, dtype: category
Categories (3, object): ['Small (< 1000)' < 'Medium (1000-2000)' < 'Large (2000-5000)']

In [305]:
 # Place the data series into a new column inside of the DataFrame
scores_by_school_size["School Size"] = pd.cut(scores_by_school_size['Size'], bins, labels=group_labels)

C:\Users\bryantim\AppData\Local\Temp\ipykernel_16624\2356858920.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scores_by_school_size["School Size"] = pd.cut(scores_by_school_size['Size'], bins, labels=group_labels)


In [306]:
# Group by spending per student group
scores_by_school_size = scores_by_school_size.groupby('School Size').mean()

# Drop budget per student column
scores_by_school_size = scores_by_school_size[
    ['Average Math Score', '% Passing Math', '% Passing Reading', '% Passing Overall']]

### Scores by School Size

In [307]:
scores_by_school_size

,Average Math Score,% Passing Math,% Passing Reading,% Passing Overall
School Size,,,,
Small (< 1000),83.821598,91.158155,92.471895,89.883853
Medium (1000-2000),83.374684,89.931303,93.244843,90.621535
Large (2000-5000),77.746417,67.631335,80.190800,58.286003


In [308]:
# Create a DataFrame from school_summary of just the columns needed
scores_by_school_type = school_summary[[
    'School Type', 'Average Math Score', '% Passing Math', '% Passing Reading', '% Passing Overall']]

In [309]:
# Group by school type
scores_by_school_type = scores_by_school_type.groupby('School Type').mean()

### Scores by School Type

In [310]:
scores_by_school_type

,Average Math Score,% Passing Math,% Passing Reading,% Passing Overall
School Type,,,,
Charter,83.473852,90.363226,93.052812,90.432244
District,76.956733,64.302528,78.324559,53.672208
